In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
import MathOptInterface as MOI
import Ipopt 
using LinearAlgebra
import ForwardDiff
using Random

  Activating environment at `C:\Users\Matthew\Desktop\Work\CMU\Spring 2023\Optimal Control\Project\Project.toml`


In [2]:
include(joinpath(@__DIR__, "utils","fmincon.jl"))
include(joinpath(@__DIR__, "utils","quadrotor.jl"))

animate_quadrotor (generic function with 1 method)

In [25]:
function skew(ω::Vector)
    return [0    -ω[3]  ω[2];
            ω[3]  0    -ω[1];
           -ω[2]  ω[1]  0]
end

function quad_payload_dynamics(model, x,u)
    L_p = model.L_p
    m_p = model.mass_payload
    m_q = model.mass_quad
    J = model.J
    g = model.gravity
    kf = model.kf
    km = model.km
    ℓ = model.ℓ
    
    r_q = x[1:3]
    q = x[4:6]
    r_l = x[7:8]
    v_l = x[9:10]
    v_q = x[11:13]
    ω = x[14:16]
    
    ṙ_q = v_q
    q̇ = ((1+norm(q)^2)/4) *(I + 2*(skew(q)^2 + skew(q))/(1+norm(q)^2))*ω
    ṙ_l = v_l
        
    B = [I; -r_l' / sqrt(L_p - r_l'*r_l)]
    
    R_ib = dcm_from_mrp(q)' # Rotation from body to world frame '
    
    # Calculate Control Thrust and Torques
    F1 = max(0,kf*u[1])
    F2 = max(0,kf*u[2])
    F3 = max(0,kf*u[3])
    F4 = max(0,kf*u[4])
    F_b = [0.; 0.; F1+F2+F3+F4] #total rotor force in body frame
    F = -R_ib * F_b # negative since z axis in body frame is down
    
    M1 = km*u[1]
    M2 = km*u[2]
    M3 = km*u[3]
    M4 = km*u[4]
    M_b = [ℓ*(F3-F1), ℓ*(F2-F4), (M1-M2+M3-M4)] #total rotor torque in body frame
    # paper has two extra terms one for air drag (we ignore) and another for gyroscopic torque
    # Zac seems to ignore gyroscopic but not sure why
            
    
    # TODO: double check Ḃ (I plugged in the scalar form into wolfram alpha and took the derivative)
    Ḃ = [0 0; 0 0; L_p^2 * v_l' / (L_p^2 - r_l' * r_l)^(3/2)]
    ω_skew = skew(ω)

    RHS = [m_p * B' * g; (m_q + m_p) * g; zeros(3)] + [zeros(2); F; M_b]    
    Const = [-m_p * B' * Ḃ * v_l; m_p * Ḃ * v_l; ω_skew * J * ω]

    A = zeros(eltype(x), 8,8)
    A[1:2, :] = [m_p*B'*B m_p*B' zeros(2,3)]
    A[3:5, :] = [m_p*B (m_q+m_p)*I(3) zeros(3,3)]
    A[6:8, :] = [zeros(3,5) J]

    derivs = A \ (RHS - Const)
    
    return [ṙ_q; q̇; ṙ_l; derivs]
end

function hermite_simpson(model::NamedTuple, x1::Vector, x2::Vector, u, dt::Real)::Vector
    # TODO: input hermite simpson implicit integrator residual 
    ẋ1 = quad_payload_dynamics(model, x1, u)
    ẋ2 = quad_payload_dynamics(model, x2, u)
    
    xk_half = (x1 + x2)/2 + dt * (ẋ1 - ẋ2) / 8
    return x1 + dt * (ẋ1 + 4 * quad_payload_dynamics(model, xk_half, u) + ẋ2) / 6 - x2
end

function create_idx(nx,nu,N)
    # This function creates some useful indexing tools for Z 
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    
    # Feel free to use/not use anything here.
    
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

function quad_cost(params::NamedTuple, Z::Vector)::Real
    idx, N, x_goals = params.idx, params.N, params.x_goals
    Q, R, Qf = params.Q, params.R, params.Qf
    
    # TODO: input cartpole LQR cost 
    
    J = 0 
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
       
        J += 1/2 * (xi - x_goals[i,:])' * Q * (xi - x_goals[i,:]) + 1/2 * ui' * R * ui

    end
    
    # dont forget terminal cost
    J += 1/2 * (Z[idx.x[N]]-  x_goals[N,:])' * Qf * (Z[idx.x[N]] - x_goals[N,:])
    
    return J 
end

function quad_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    model_w_payload, model_wo_payload = params.model_w_payload, params.model_wo_payload
    M1, M2 = params.M1, params.M2
    
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)
    
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]] 
        xip1 = Z[idx.x[i+1]]
        
        if (i in M1)
            c[idx.c[i]] = hermite_simpson(model_w_payload, xi, xip1, ui, dt)
        else 
            c[idx.c[i]] = hermite_simpson(model_wo_payload, xi, xip1, ui, dt)                
        end
    end
    return c 
end

function quad_equality_constraint(params::NamedTuple, Z::Vector)::Vector
    N, idx, xic, x_goals = params.N, params.idx, params.xic, params.x_goals
    
    # TODO: return all of the equality constraints 
    eq_constraints = [
        Z[idx.x[1]] - xic; 
        Z[idx.x[N÷2]] - x_goals[1,:]
        Z[idx.x[N]] - x_goals[N,:]; quad_dynamics_constraints(params, Z)]
    
    return eq_constraints
end

function solve_quad(params;verbose=true)
    idx = params.idx
    
    # TODO: primal bounds 
    x_l = -Inf * ones(idx.nz)
    x_u =  Inf * ones(idx.nz)
    
    for i = 1 : N - 1
        x_l[idx.x[i][7:8]] .= -0.5
        x_u[idx.x[i][7:8]] .= 0.5
    end
    
    # inequality constraint bounds (this is what we do when we have no inequality constraints)
    c_l = zeros(0)
    c_u = zeros(0)
    function inequality_constraint(params, Z)
        return zeros(eltype(Z), 0)
    end
    
    # initial guess 
    z0 = 0.001*randn(idx.nz)
    
    # choose diff type (try :auto, then use :finite if :auto doesn't work)
    diff_type = :auto 
#     diff_type = :finite
    
    
    Z = fmincon(quad_cost, quad_equality_constraint,inequality_constraint,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-6, c_tol = 1e-6, max_iters = 100, verbose = verbose)
    
    # pull the X and U solutions out of Z 
    X = [Z[idx.x[i]] for i = 1:N]
    U = [Z[idx.u[i]] for i = 1:(N-1)]
    
    return X, U, t_vec, params 
end

solve_quad (generic function with 1 method)

In [26]:
# problem size 
nx = 16
nu = 4
dt = 0.1
tf = 8.0 
t_vec = 0:dt:tf 
N = length(t_vec)

# LQR cost 
Q = diagm(ones(nx))
R = 0.1*diagm(ones(nu))
Qf = 10*diagm(ones(nx))

# indexing 
idx = create_idx(nx,nu,N)

# initial and goal states 
xic = zeros(nx)

x_mid = zeros(nx)
x_mid[1] = -1
x_mid[2] = -1
x_mid[3] = -1

x_goals = zeros(N,nx)
for i = 1:N÷2
    x_goals[i,:] = x_mid
end

x_end = zeros(nx)
x_end[1] = -2
x_end[2] = -2
x_end[3] = 0
for i = N÷2:N
    x_goals[i,:] = x_end
end

traj_length = N÷2
M1 = vcat([ (i-1)*10      .+ (1:traj_length)   for i = 1:1]...) # stack the set into a vector
M2 = vcat([((i-1)*10 + traj_length) .+ (1:traj_length)   for i = 1:1]...) # stack the set into a vector

# load all useful things into params 
model_w_payload = (mass_quad=0.5,
        mass_payload=0.5,
        ℓ = 0.1750, # Quad radius
        L_p = 1, # Pendulum length
        J=Diagonal([0.0023, 0.0023, 0.004]),
        gravity=[0;0;9.81],
        kf=1.0,
        km=0.0245, dt = dt)
model_wo_payload = (mass_quad=0.5,
        mass_payload=0.1,
        ℓ = 0.1750, # Quad radius
        L_p = 1, # Pendulum length
        J=Diagonal([0.0023, 0.0023, 0.004]),
        gravity=[0;0;9.81],
        kf=1.0,
        km=0.0245, dt = dt)    

params = (
        Q = Q, 
        R = R, 
        Qf = Qf, 
        xic = xic,
        x_goals = x_goals,
        dt = dt, 
        N = N, 
        idx = idx, 
        model_w_payload=model_w_payload,
        model_wo_payload=model_wo_payload,
        M1=M1,
        M2=M2)

X, U, t_vec = solve_quad(params, verbose=true);

---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :auto (ForwardDiff.jl)----
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------
This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:  2146048
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:     1616
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      160
                     variables with only upper bounds:        0
Total number of equality constraints.................:     1328
Total number of inequality constraints.......

In [22]:
import MeshCat as mc
using CoordinateTransformations, Rotations, Colors
using GeometryBasics
import RobotDynamics as RD
using StaticArrays

function set_mesh!(vis, length)
    
    obj = joinpath(@__DIR__, "utils/quadrotor.obj")
    
    robot_obj = mc.MeshFileGeometry(obj)
    quad_mat = mc.MeshPhongMaterial(color=colorant"black")
    
    pole_mat = mc.MeshPhongMaterial(color=colorant"blue")
    mass_mat = mc.MeshPhongMaterial(color=colorant"red")
    pole = mc.Cylinder(mc.Point3f0(0,0,0),Point3f0(0,0,-length),0.01f0)
    mass = mc.HyperSphere(mc.Point3f0(0,0,0), 0.05f0)
    
    mc.setobject!(vis["quad"], robot_obj, quad_mat)
    mc.setobject!(vis["pole"], pole, pole_mat)
    mc.setobject!(vis["mass"], mass, mass_mat)
end
#Quaternion stuff
function hat(v)
    return [0 -v[3] v[2];
            v[3] 0 -v[1];
            -v[2] v[1] 0]
end
function L(q)
    s = q[1]
    v = q[2:4]
    L = [s    -v';
         v  s*I+hat(v)]
    return L
end
T = Diagonal([1; -ones(3)])
H = [zeros(1,3); I]
function qtoQ(q)
    return H'*T*L(q)*T*L(q)*H
end
function G(q)
    G = L(q)*H
end
function rptoq(ϕ)
    (1/sqrt(1+ϕ'*ϕ))*[1; ϕ]
end
function qtorp(q)
    q[2:4]/q[1]
end

function visualize!(vis, x::StaticVector, attached, last_pos)
    quad_rot_mat = dcm_from_mrp(x[4:6])

    # NED = North, East, Down (coord frame used in paper)
    # ENU = East, North, Up (coord frame used by meshcat)
    quad_ned = x[1:3]
    quad_enu = [quad_ned[2]; quad_ned[1]; -quad_ned[3]]
    
    mass_ned = [x[8] + quad_ned[1]; x[9] + quad_ned[2]; sqrt(L_p^2 - x[7:8]' * x[7:8]) + quad_ned[3]]
    mass_enu = [mass_ned[2]; mass_ned[1]; -mass_ned[3]]
    
    # Find rotation that points a vector from quad to mass
    up = [0;0;1]
    dir = quad_enu - mass_enu
    q = [0; cross(up, dir)]
    q[1] = sqrt(norm(up)^2 * norm(dir)^2) + dot(up, dir)
    q = normalize(q)
    pole_rot_mat = qtoQ(q)
    
    mc.settransform!(vis["quad"], compose(Translation(quad_enu...), LinearMap(quad_rot_mat)))
    mc.settransform!(vis["pole"], compose(Translation(quad_enu...), LinearMap(pole_rot_mat)))
    
    
    if (attached)
        mc.settransform!(vis["mass"], compose(Translation(mass_enu...)))
        return mass_enu
    else
        mc.settransform!(vis["mass"], compose(Translation(last_pos...)))
        return last_pos
    end
    
end

function visualize!(vis, tf::Real, X, params)
    fps = Int(round((length(X)-1)/tf))
    anim = mc.Animation(fps)
    x_dim = size(X[1])[1]
    last_pos = [0;0;0]
    
    for (k,x) in enumerate(X)
        mc.atframe(anim, k) do
            x = X[k]
            
            if k in params.M1
                last_pos = visualize!(vis, SVector{x_dim}(x), true, last_pos) 
            else
                visualize!(vis, SVector{x_dim}(x), false, last_pos) 
            end
        end
    end
    mc.setanimation!(vis, anim)
end

visualize! (generic function with 2 methods)

In [19]:
vis = mc.Visualizer()
mc.render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
└ http://127.0.0.1:8701


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8701))

In [27]:
set_mesh!(vis, params.model_w_payload.L_p)

x_dim = size(X[1])[1]
X1 = [SVector{x_dim}(x) for x in X];
visualize!(vis, t_vec[end], X1, params)